## Methodology for preparing geojson of the upper midwest states with data for onclick events

1. Start with US geojson from US Census
2. Bring into https://geojson.io and delete all the boundaries not wanted, save that file (midwest.geojson)
3. Prepare the csv file with the data for the onclick events (map_data.csv)
4. Read the geojson and the csv file into Pandas
5. Merge those two files using geopandas
6. Export the merge df as a shape file using geopandas
7. Convert the shape files to geojson using an online converter (https://mygeodata.cloud/converter/shp-to-geojson).

Didn't use Dom's code because I couln't get the geometry to export. Left code at the bottom on notebook for future reference.

In [1]:
import pandas as pd
import geopandas
import json
import numpy as np
import fiona
# import fiona; help(fiona.open)

In [2]:

gdf = geopandas.read_file("US_map/midwest.geojson")
pdf = pd.read_csv("map_data.csv")

In [3]:
gdf

,AREA,PERIMETER,ST99_D00_,ST99_D00_I,STATE,NAME,LSAD,REGION,DIVISION,LSAD_TRANS,geometry
0,25.539497,31.407183,83,82,27,Minnesota,01,2,4,None,"POLYGON ((-95.15332 49.19311, -95.15342 49.249..."
1,21.839790,21.589482,88,87,38,North Dakota,01,2,4,None,"POLYGON ((-97.42517 49.00051, -97.41122 49.000..."
2,16.456982,22.984328,103,102,55,Wisconsin,01,2,3,None,"POLYGON ((-90.41814 46.56609, -90.41562 46.563..."
3,22.578395,23.366282,106,105,46,South Dakota,01,2,4,None,"POLYGON ((-104.04544 45.94531, -103.92387 45.9..."
4,15.857152,20.216172,139,138,19,Iowa,01,2,4,None,"POLYGON ((-91.36933 43.50083, -91.29205 43.500..."


In [4]:
pdf

,NAME,State,AltFuelStations,ChargeStations,ChargeOutlets,2005Emissions(MtCO2),2005EmissionsReduce,2005EmissionsTrans(MtCO2),2005EmissionsTransReduce,Incentives,Population,MedianIncome
0,Iowa,IA,571,127,213,129.508525,64.754262,21.804555,10.902277,5,3148618,68718
1,Minnesota,MN,1112,326,793,133.031986,66.515993,37.647662,18.823831,6,5606249,71817
2,North Dakota,ND,100,19,29,66.806329,33.403165,6.419279,3.209640,7,758080,66505
3,South Dakota,SD,132,33,89,36.593148,18.296574,6.257833,3.128917,8,878698,59463
4,Wisconsin,WI,699,285,542,135.823441,67.911720,31.331999,15.666000,9,5807406,62629


In [5]:
joined_gdf = gdf.merge(pdf, on="NAME")
joined_gdf

,AREA,PERIMETER,ST99_D00_,ST99_D00_I,STATE,NAME,LSAD,REGION,DIVISION,LSAD_TRANS,...,AltFuelStations,ChargeStations,ChargeOutlets,2005Emissions(MtCO2),2005EmissionsReduce,2005EmissionsTrans(MtCO2),2005EmissionsTransReduce,Incentives,Population,MedianIncome
0,25.539497,31.407183,83,82,27,Minnesota,01,2,4,None,...,1112,326,793,133.031986,66.515993,37.647662,18.823831,6,5606249,71817
1,21.839790,21.589482,88,87,38,North Dakota,01,2,4,None,...,100,19,29,66.806329,33.403165,6.419279,3.209640,7,758080,66505
2,16.456982,22.984328,103,102,55,Wisconsin,01,2,3,None,...,699,285,542,135.823441,67.911720,31.331999,15.666000,9,5807406,62629
3,22.578395,23.366282,106,105,46,South Dakota,01,2,4,None,...,132,33,89,36.593148,18.296574,6.257833,3.128917,8,878698,59463
4,15.857152,20.216172,139,138,19,Iowa,01,2,4,None,...,571,127,213,129.508525,64.754262,21.804555,10.902277,5,3148618,68718


In [6]:
list(joined_gdf.columns)

['AREA',
 'PERIMETER',
 'ST99_D00_',
 'ST99_D00_I',
 'STATE',
 'NAME',
 'LSAD',
 'REGION',
 'DIVISION',
 'LSAD_TRANS',
 'geometry',
 'State',
 'AltFuelStations',
 'ChargeStations',
 'ChargeOutlets',
 '2005Emissions(MtCO2)',
 '2005EmissionsReduce',
 '2005EmissionsTrans(MtCO2)',
 '2005EmissionsTransReduce',
 'Incentives',
 'Population',
 'MedianIncome']

In [7]:
# # Try writing to file using geopandas

# joined_gdf.to_file("test.geojson", driver="GeoJSON")

In [8]:
# Write to shape file

joined_gdf.to_file("midwest_final.shp")

C:\Users\khagl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


## Code from Dom


In [9]:
# # Define a helper function to format Geojson
# # Dom worked to customize this to convert data to dictionary

# def dfGeoJSON(joined_gdf):
#     geojson = {'type': 'FeatureCollection', 'features':[]}
    
#     for i in range(len(joined_gdf)):
        
#         dict = {}
#         dict['AREA'] = joined_gdf.iloc[i, 0]
#         dict['PERIMETER'] = joined_gdf.iloc[i, 1]
#         dict['NAME'] = joined_gdf.iloc[i, 5]
#         dict['geometry'] = joined_gdf.iloc[i, 10]
#         dict['State'] = joined_gdf.iloc[i, 11]
#         dict['AltFuelStations'] = joined_gdf.iloc[i, 12]
#         dict['ChargeStations'] = joined_gdf.iloc[i, 13]
#         dict['ChargeOutlets'] = joined_gdf.iloc[i, 14]
#         dict['2005Emissions(MtCO2)'] = joined_gdf.iloc[i, 15]
#         dict['2005EmissionsTrans(MtCO2)	'] = joined_gdf.iloc[i, 17]
#         dict['Incentives'] = joined_gdf.iloc[i, 19]
        
#         # feature = joined_gdf.iloc[i, 2]
#         geojson['features'].append(dict)
        
#     return geojson

In [10]:
# # Convert the updated dataframe back to a GeoJSON

# geojson = dfGeoJSON(joined_gdf)

In [11]:
#geojson

In [12]:
# # Define a helper function to deal with numpy datatypes

# def convert(o):
#     if isinstance(o, np.int64): return int(o)
# #     raise TypeError

In [13]:
# # Save the GeoJSON file

# with open("final_geojson.geojson", 'w') as file:
#     file.write(json.dumps(geojson, default=convert))

In [14]:
# Original code Dom provided

# # Define a helper function to format a GeoJSON
# def dfGeoJSON(df):
#     geojson = {'type': 'FeatureCollection', 'features': []}
#     for i in range(len(df)):
#         feature = df.iloc[i, 1]
#         geojson['features'].append(feature)
#     return geojson

# # Convert the updated dataframe back to a GeoJSON
# geojson = dfGeoJSON(df)

# # Define a helper function to deal with numpy datatypes
# def convert(o):
#     if isinstance(o, np.int64): return int(o)  
#     raise TypeError
    
# # Finally, save the new GeoJSON file
# with open(new_geojson_file, 'w') as file:
#      file.write(json.dumps(geojson, default=convert))    